# Single Year Function:

this function get_migration_data reads two csv files containing migration data for all US states in a given year and returns the net migration for each state

  Parameters:
  - 'csv_link_outflow' (url): link to outflow data
  - 'csv_link_inflow' (url): link to inflow data

  Returns:
  - 'migration_data': (dictionary): dict of state abbreviations as keys and their respective net migration value (between -1 and 1) as values




## Accessing Data
- There might be a better way to do this thank copying the links but idk
- I get them from [this website](https://www.irs.gov/statistics/soi-tax-stats-migration-data)
- Click on the year, the data used for this function is the state inflow and outflow data
- below, the outflow2021 and inflow2021 are the 2020-2021 data


In [ ]:
import pandas as pd

In [ ]:
def get_migration_data(csv_link_outflow, csv_link_inflow):
  prop_migration_by_state = {}

  outflowData = pd.read_csv(csv_link_outflow)
  inflowData = pd.read_csv(csv_link_inflow)

  outflowStates_dfs = {}
  inflowStates_dfs = {}

  for state in outflowData['y2_state'].unique():
    if state not in ['DC', 'FR']:
        #create new df for state
        outflowStates_dfs[state] = outflowData[outflowData['y2_state'] == state].copy()
        inflowStates_dfs[state] = inflowData[inflowData['y1_state'] == state].copy()

        #calculate total pop
        total_pop = 0
        matching_rows = outflowStates_dfs[state][outflowStates_dfs[state]['y2_state_name'].str.contains('Non-migrants|Migration-Same|Migration-US')]
        total_pop = matching_rows['n1'].sum() + matching_rows['n2'].sum()
        #calculate total leaving
        leaving = 0
        non_matching_rows = outflowStates_dfs[state][~outflowStates_dfs[state]['y2_state_name'].str.contains('Non-migrants|Migration-Same|Migration-US')]
        leaving = non_matching_rows['n1'].sum() + non_matching_rows['n2'].sum()
        #calculate total incoming
        incoming = 0
        non_matching_rows = inflowStates_dfs[state][~inflowStates_dfs[state]['y1_state_name'].str.contains('Non-migrants|Migration-Same|Migration-US')]
        incoming = non_matching_rows['n1'].sum() + non_matching_rows['n2'].sum()

        #calculate prop and standardize outliers
        #   (-100) im not sure why but the proportions ended up negative based on my calculations in R and outside research regarding US state migration
        #          so im multiplying by -100 instead of 100
        #          ex. California has a negative population change from 2020-2021 but without the -100 it's proportion ends up positive 0.98
        prop = ((incoming - leaving) / total_pop) * -100

        #add to dictionary
        prop_migration_by_state[state] = prop

  return prop_migration_by_state




## Test Function for 2020-2021 data


In [ ]:
inflowTest = 'https://docs.google.com/spreadsheets/d/1felwD4GxWI3zmiLi9fNjCtjV9l1fyGucpqbmn2i3Ycc/export?format=csv'
inflowData = pd.read_csv(inflowTest)

print(inflowData.head())
print(inflowData.tail())

   y2_statefips  y1_statefips y1_state y1_state_name      n1       n2  \
0             1            63       XX       Alabama  47,503  106,325   
1             1            13       Ga       Georgia   6,574   14,353   
2             1            12       Fl       Florida   6,440   13,881   
3             1            57       FR       Foreign   3,350    7,631   
4             1            48       Tx         Texas   3,107    7,479   

         AGI  
0  1,308,623  
1    166,510  
2    154,216  
3     81,179  
4     98,292  
      y2_statefips  y1_statefips y1_state         y1_state_name       n1  \
2750            56            54       WV         West Virginia       25   
2751            56            10       De              Delaware       15   
2752            56            44       RI          Rhode Island       14   
2753            56            11       DC  District of Columbia       14   
2754            56            56     Wy S      tate Non-Migrant  159,022   

           n2 

# MIGRATION FUNCTION:

## Function description
this function create_migration_file propmts for input of one csv file containing a list of two url links (one for inflow and one for outflow) for each year from 1990-2021 regarding state migration data from the IRS

  Parameters:
  - 'list_of_links' (csv): csv file of links for each year

  Returns:
  - 'fileName': (.json): JSON file of a dictionary for each year holding migration proportions for each US state

In [ ]:
from google.colab import files
import csv
import json

In [ ]:
def create_migration_file():

  print("Please upload the CSV file containing migration data links.")
  uploaded = files.upload()

  csv_filename = next(iter(uploaded))


  # seperate years data
  # for each year, get inflow and outflow data

  #FILL IN CODE HERE
  yearsLinks = {}
  with open(csv_filename, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
      end_year = int(row['end_year'])
      inflow_link = row['inflow']
      outflow_link = row['outflow']
      yearsLinks[end_year] = (inflow_link, outflow_link)

  combined_migration_data = {}
  # call migration function
  for year, (inflow_link, outflow_link) in yearsLinks.items():
    migration_data = get_migration_data(outflow_link, inflow_link)
    combined_migration_data[year] = migration_data


  return json.dumps(combined_migration_data)

## Test function


In [ ]:
from pprint import pprint

file = create_migration_file()
pprint(json.loads(file))


Please upload the CSV file containing migration data links.


Saving copy_migr_urls - Sheet1.csv to copy_migr_urls - Sheet1 (1).csv
{'2012': {'AK': 0.6289042767228464,
          'AL': 0.11171711284122292,
          'AR': 0.07711809047831361,
          'AZ': -0.40927892539226096,
          'CA': 0.07397738977410624,
          'CO': -0.6038666492658099,
          'CT': 0.5048532877045562,
          'DE': -0.21401844703401565,
          'FL': -0.2197239199954586,
          'GA': -0.0128539298400677,
          'HI': -0.04777237778546817,
          'IA': 0.10160182323800424,
          'ID': -0.026970023360715797,
          'IL': 0.5288109948304123,
          'IN': 0.14024410203744392,
          'KS': 0.31860935883752384,
          'KY': 0.06003286480024064,
          'LA': 0.005143364013084884,
          'MA': 0.26923603443287586,
          'MD': 0.1098116266078713,
          'ME': -0.0154374052968189,
          'MI': 0.27325006552146086,
          'MN': 0.16793017726033013,
          'MO': 0.1955691611831806,
          'MS': 0.23365970505823228,
    

# Final Function

## Use create_migration_file() and upload csv to get final json file

In [ ]:
import pandas as pd
from google.colab import files
import csv
import json

In [ ]:
def get_migration_data(csv_link_outflow, csv_link_inflow):
  prop_migration_by_state = {}

  outflowData = pd.read_csv(csv_link_outflow)
  inflowData = pd.read_csv(csv_link_inflow)

  outflowStates_dfs = {}
  inflowStates_dfs = {}

  for state in outflowData['y2_state'].unique():
    if state not in ['DC', 'FR']:
        #create new df for state
        outflowStates_dfs[state] = outflowData[outflowData['y2_state'] == state].copy()
        inflowStates_dfs[state] = inflowData[inflowData['y1_state'] == state].copy()

        #calculate total pop
        total_pop = 0
        matching_rows = outflowStates_dfs[state][outflowStates_dfs[state]['y2_state_name'].str.contains('Non-migrants|Migration-Same|Migration-US')]
        total_pop = matching_rows['n1'].sum() + matching_rows['n2'].sum()
        #calculate total leaving
        leaving = 0
        non_matching_rows = outflowStates_dfs[state][~outflowStates_dfs[state]['y2_state_name'].str.contains('Non-migrants|Migration-Same|Migration-US')]
        leaving = non_matching_rows['n1'].sum() + non_matching_rows['n2'].sum()
        #calculate total incoming
        incoming = 0
        non_matching_rows = inflowStates_dfs[state][~inflowStates_dfs[state]['y1_state_name'].str.contains('Non-migrants|Migration-Same|Migration-US')]
        incoming = non_matching_rows['n1'].sum() + non_matching_rows['n2'].sum()

        #calculate prop and standardize outliers
        #   (-100) im not sure why but the proportions ended up negative based on my calculations in R and outside research regarding US state migration
        #          so im multiplying by -100 instead of 100
        #          ex. California has a negative population change from 2020-2021 but without the -100 it's proportion ends up positive 0.98
        prop = ((incoming - leaving) / total_pop) * -100

        #add to dictionary
        prop_migration_by_state[state] = prop

  return prop_migration_by_state


In [ ]:
def create_migration_file():

  print("Please upload the CSV file containing migration data links.")
  uploaded = files.upload()

  csv_filename = next(iter(uploaded))


  # seperate years data
  # for each year, get inflow and outflow data

  #FILL IN CODE HERE
  yearsLinks = {}
  with open(csv_filename, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
      end_year = int(row['end_year'])
      inflow_link = row['inflow']
      outflow_link = row['outflow']
      yearsLinks[end_year] = (inflow_link, outflow_link)

  combined_migration_data = {}
  # call migration function
  for year, (inflow_link, outflow_link) in yearsLinks.items():
    migration_data = get_migration_data(outflow_link, inflow_link)
    combined_migration_data[year] = migration_data


  return json.dumps(combined_migration_data)